In [2]:
import timm

model = timm.create_model('resnet101', pretrained=True)
model.relu

AttributeError: 'ResNet' object has no attribute 'relu'

In [5]:
from omegaconf import OmegaConf
import torch.nn.functional as F
import torch

cfg = OmegaConf.load('configs/apn_CUB.yaml')
list(name.lower() for name in cfg.MODEL.LOSSES.keys())

['l_cls', 'l_reg', 'l_cpt']

In [31]:
from scipy import stats
def update_prior_dist(batch_size, alpha, beta):
    grid_points = torch.arange(1., 2*batch_size, 2.).float() / (2*batch_size)
    print(grid_points)
    grid_points_np = grid_points.cpu().numpy()
    grid_points_icdf = stats.beta.ppf(grid_points_np, a=alpha, b=beta)
    print(grid_points_icdf)
    prior_dist = torch.tensor(grid_points_icdf).float().unsqueeze(1)
    return prior_dist

In [32]:
prior = update_prior_dist(32, 1, 1e-3)

tensor([0.0156, 0.0469, 0.0781, 0.1094, 0.1406, 0.1719, 0.2031, 0.2344, 0.2656,
        0.2969, 0.3281, 0.3594, 0.3906, 0.4219, 0.4531, 0.4844, 0.5156, 0.5469,
        0.5781, 0.6094, 0.6406, 0.6719, 0.7031, 0.7344, 0.7656, 0.7969, 0.8281,
        0.8594, 0.8906, 0.9219, 0.9531, 0.9844])
[0.99999986 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]


In [27]:
import numpy as np
sum(prior)

tensor([32.])

In [23]:
prior.shape

AttributeError: 'torch.Size' object has no attribute 'shape'

In [12]:
import torch
b = 16
grid = torch.arange(1., 2*b, 2.).numpy() / 2 * b

In [13]:


stats.beta.ppf(grid, a=1, b=1e-3)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan])

In [8]:
x = torch.randn(4, 2048, 7,7)
F.conv2d(x, torch.randn(100, 2048, 1, 1)).shape

torch.Size([4, 100, 7, 7])

In [1]:
import torch
from omegaconf import OmegaConf
import  torch.nn.functional as f
from torchvision.models import resnet101, ResNet101_Weights

from apn import BackBone

In [8]:
import timm
model = timm.create_model('resnet101', pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [6]:
import os
from pathlib import Path
os.path.basename('configs/resnet101_ft_CUB.yaml')

'backbone_ft_CUB.yaml'

In [7]:
Path('configs/resnet101_ft_CUB.yaml').stem

'backbone_ft_CUB'

In [3]:
backbone = BackBone('resnet101', num_classes=200)

In [4]:
set(backbone.state_dict().keys()) == set(resnet101(weights=ResNet101_Weights.DEFAULT).state_dict().keys())

True

In [26]:
conf = OmegaConf.load('configs/resnet101_ft_CUB.yaml')

In [30]:
type(conf.MODEL.LR)

float

In [18]:
model = resnet101(weights=ResNet101_Weights.DEFAULT)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=200)

In [20]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 